In [1]:
import pandas as pd
import numpy as np


In [2]:
df=pd.read_csv('C:/Users/aakas/Desktop/Downloads/Chrome Downloads/output1.csv')

In [3]:
test_data_preprocessed=pd.read_csv("C:/Users/aakas/Desktop/Downloads/Chrome Downloads/datatotest.csv")

In [10]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import Sequence
from scipy.sparse import vstack

# Drop rows with NaN values in 'review_text' column
df_input = df[['review_text', 'n_votes', 'n_comments', 'rating']]
df_input = df_input.dropna(subset=['review_text'])

# Split the data into training and testing sets
train_data, test_data, train_labels, test_labels = train_test_split(
    df_input[['review_text', 'n_votes', 'n_comments']], df_input['rating'],
    test_size=0.2, random_state=0
)

# TF-IDF vectorization
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf_vectorizer.fit_transform(train_data['review_text'])
X_test_tfidf = tfidf_vectorizer.transform(test_data['review_text'])

# Convert labels to numerical format
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(train_labels)
y_test = label_encoder.transform(test_labels)

# Build a custom generator to handle sparse input
class SparseSequence(Sequence):
    def __init__(self, X, y, batch_size):
        self.X = X
        self.y = y
        self.batch_size = batch_size
        self.n_samples, self.n_features = X.shape

    def __len__(self):
        return int(np.ceil(self.n_samples / self.batch_size))

    def __getitem__(self, idx):
        start_idx = idx * self.batch_size
        end_idx = (idx + 1) * self.batch_size
        batch_X = self.X[start_idx:end_idx].toarray()  # Convert sparse matrix to NumPy array
        batch_y = self.y[start_idx:end_idx]
        return batch_X, batch_y

# Build the deep neural network model
model = Sequential()
model.add(Dense(256, input_dim=X_train_tfidf.shape[1], activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(len(np.unique(y_train)), activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model using the custom generator
batch_size = 32
train_generator = SparseSequence(X_train_tfidf, y_train, batch_size=batch_size)
validation_generator = SparseSequence(X_test_tfidf, y_test, batch_size=batch_size)

# Train the model
model.fit(train_generator, epochs=6, validation_data=validation_generator)


Epoch 1/6
15749/15749 [==============================] - 170s 11ms/step - loss: 1.1102 - accuracy: 0.5197 - val_loss: 1.0663 - val_accuracy: 0.5376
Epoch 2/6
15749/15749 [==============================] - 160s 10ms/step - loss: 1.0435 - accuracy: 0.5492 - val_loss: 1.0597 - val_accuracy: 0.5413
Epoch 3/6
15749/15749 [==============================] - 163s 10ms/step - loss: 0.9868 - accuracy: 0.5743 - val_loss: 1.0691 - val_accuracy: 0.5366
Epoch 4/6
15749/15749 [==============================] - 162s 10ms/step - loss: 0.9248 - accuracy: 0.6054 - val_loss: 1.0914 - val_accuracy: 0.5345
Epoch 5/6
15749/15749 [==============================] - 162s 10ms/step - loss: 0.8686 - accuracy: 0.6325 - val_loss: 1.1321 - val_accuracy: 0.5292
Epoch 6/6
15749/15749 [==============================] - 162s 10ms/step - loss: 0.8193 - accuracy: 0.6570 - val_loss: 1.1620 - val_accuracy: 0.5242


In [11]:
test_data_preprocessed['review_text'].fillna('', inplace=True)


In [12]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import tensorflow as tf

# Assuming your preprocessed test data has columns 'review_text', 'n_votes', 'n_comments'
X_test_preprocessed_tfidf = tfidf_vectorizer.transform(test_data_preprocessed['review_text'])

# Convert the SciPy sparse matrix to SparseTensor
X_test_preprocessed_tfidf_coo = X_test_preprocessed_tfidf.tocoo()
indices = np.column_stack((X_test_preprocessed_tfidf_coo.row, X_test_preprocessed_tfidf_coo.col))
values = X_test_preprocessed_tfidf_coo.data
dense_shape = X_test_preprocessed_tfidf_coo.shape

X_test_preprocessed_tfidf_sparse = tf.SparseTensor(indices, values, dense_shape)

# Reorder sparse matrix indices using tf.sparse.reorder
X_test_preprocessed_tfidf_reordered = tf.sparse.reorder(X_test_preprocessed_tfidf_sparse)

# Use the trained model to predict labels for the preprocessed test data
y_pred = np.argmax(model.predict(X_test_preprocessed_tfidf_reordered), axis=-1)

# Map the predicted labels back to their original classes using label_encoder
predicted_labels = label_encoder.inverse_transform(y_pred)

# Print or save the predicted labels
print(predicted_labels)


8438/8438 [==============================] - 21s 2ms/step
[4 5 3 ... 3 3 5]


In [13]:
predicted_labels.shape


(270000,)

In [14]:
import pandas as pd

# Assuming df_test is your original DataFrame and 'review_id' is a column in it
# If df_test is already a DataFrame, you might want to access 'review_id' directly.
# If df_test is not a DataFrame, replace df_test with your actual DataFrame.

df1 = test_data_preprocessed['review_id'].reset_index(drop=True)  # Reset index
df2 = pd.Series(predicted_labels, name='Prediction')   # Convert NumPy array to Pandas Series

# Merge the DataFrames using pd.concat
merged_df = pd.concat([df1, df2], axis=1)

# Display the merged DataFrame
print(merged_df)

# Save the merged DataFrame to a new CSV file
merged_df.to_csv('merged_data3.csv', index=False)


                               review_id  Prediction
0       583b1ad3d3ab9b0f967bc73c2f5d68d8           4
1       e53e941fb1bd2f76fb270cedbb1479b8           5
2       e2ab3d609db0a5c7aaed9ee136073c53           3
3       740ebae0f5b7fd0b7c9890ef306099f6           2
4       f4812f3464aaf50d56480025afb22c57           3
...                                  ...         ...
269995  1bccc443a9a97a3fd0b45646f39b6465           3
269996  ea52067f73ba6de7563eaf4567d423ef           5
269997  153121e003a1d19db89d45723dd0ad8e           3
269998  ce89b8c499252125f8983ef04a0d4ad9           3
269999  454e60cbf37ec1479d0e2438cab10108           5

[270000 rows x 2 columns]


In [15]:
df

,user_id,book_id,review_id,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments,rating
0,d9a0f14b836e2634b89a6a7d4d9aa184,23308084,7bde8725cbb9ceb697c72c12262dff53,complaint not trilogyi want fantastic end rene...,Thu Jul 23 22:27:34 -0700 2015,Wed May 04 20:04:40 -0700 2016,Wed May 04 00:00:00 -0700 2016,Mon Apr 25 00:00:00 -0700 2016,1,1,5
1,3af7e1cda1d80d6a6d73e06eab301368,23310751,a9aa8356ef1ca470c98712e17005517b,read 2 days ago remember nothing disappoint,Thu Oct 13 19:33:58 -0700 2016,Thu Oct 13 19:34:28 -0700 2016,Mon Oct 10 00:00:00 -0700 2016,NaN,3,0,2
2,dd669721e136c1be47d739b14fa23d20,210252,8739baeb543858142605442041d79524,not fan first monster blood second addition pl...,Thu Feb 27 01:44:54 -0800 2014,Sun Oct 18 20:46:53 -0700 2015,Thu Feb 27 00:00:00 -0800 2014,NaN,0,0,2
3,aec624fd1ad0034f2553b7dc55ee1cd0,472392,1791472bb94c9733802303ecf34d9c53,love theo much interaction brother absolutely ...,Sun Aug 28 08:19:18 -0700 2016,Sat Sep 03 17:42:00 -0700 2016,Fri Sep 02 03:12:59 -0700 2016,Sun Aug 28 00:00:00 -0700 2016,0,0,4
4,d67aef0988e1814a819259eb11c92788,17675462,d98212782db1271607a94c5836ef6189,good book get middle part look forward read se...,Mon Jan 11 07:20:47 -0800 2016,Mon Mar 14 07:30:27 -0700 2016,Sun Mar 13 00:00:00 -0800 2016,Mon Jan 11 00:00:00 -0800 2016,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...
629995,34d07fb9e04bfbbb17371435223b120e,296182,3de0c7af9700e937b488e129ca3c9cc9,edit 112613 advice regard vorkosigan saga boil...,Sat Jul 06 23:27:10 -0700 2013,Thu Jul 10 23:35:49 -0700 2014,NaN,NaN,8,3,4
629996,c3d7ca2c133140d684ab4d958d5e4ee9,5776788,96a1196bfce7fc2d4d4e589d990badc9,not book would likely pick read get via nook f...,Tue Oct 19 17:38:21 -0700 2010,Wed Dec 22 09:46:33 -0800 2010,Tue Dec 21 00:00:00 -0800 2010,Sat Dec 18 00:00:00 -0800 2010,0,0,3
629997,60982541be85a0611e9634b4f63d0cb0,3648,a857718ca7e70b8c0ffc5ead14512fb8,book summary boast spectacular story write fla...,Thu Jun 08 22:25:07 -0700 2017,Thu Jun 08 22:28:54 -0700 2017,Fri Jun 09 07:16:37 -0700 2017,NaN,13,0,2
629998,b43eaf1760e1b11bc224815a3f3c48a3,13507212,393045562fb081cf0a6975a2f6b91908,excellent sequel wolf hall cover shorter perio...,Sun Jan 08 08:38:38 -0800 2017,Sun Jan 15 06:16:15 -0800 2017,NaN,NaN,0,0,5
